In [1]:
import duckdb

In [2]:
duck = duckdb.connect()

In [3]:
duck.sql("select try_cast(Id as int64), What_Id, Who_id from read_csv('data/Tasks - Tasks.csv')")

┌────────────────────────┬────────────────────┬────────────────────┐
│ TRY_CAST(Id AS BIGINT) │      What_Id       │       Who_Id       │
│         int64          │      varchar       │      varchar       │
├────────────────────────┼────────────────────┼────────────────────┤
│     386758000061192211 │ 386758000061193001 │ NULL               │
│     386758000061192185 │ 386758000061192445 │ 386758000061192440 │
│     386758000061148026 │ 386758000061124693 │ 386758000061124688 │
│     386758000061151021 │ 386758000060315001 │ NULL               │
│     386758000061147032 │ 386758000056967806 │ 386758000054274376 │
│     386758000061181003 │ 386758000054274381 │ 386758000054274376 │
│     386758000061178003 │ 386758000060590663 │ 386758000060590658 │
│     386758000061165003 │ 386758000058411726 │ NULL               │
│     386758000061125003 │ 386758000060763002 │ NULL               │
│     386758000061146001 │ 386758000060910001 │ NULL               │
│              ·         │        

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Install and load the postgres extension
duck.install_extension('postgres')
duck.load_extension('postgres')

# Database configuration
db_host = os.getenv('DB_HOST', 'localhost')
db_port = os.getenv('DB_PORT', '5432')
db_name = os.getenv('DB_NAME', 'postgres')
db_user = os.getenv('DB_USER', 'postgres')
db_password = os.getenv('DB_PASSWORD', '')

# Attach PostgreSQL database to DuckDB
duck.sql(f"""
    ATTACH 'host={db_host} port={db_port} dbname={db_name} user={db_user} password={db_password}' 
    AS postgres_db (TYPE POSTGRES)
""")

print(f"✓ Successfully connected DuckDB to PostgreSQL database '{db_name}'")


✓ Successfully connected DuckDB to PostgreSQL database 'medisoft'


In [5]:
duck.sql("select count(*) from postgres_db.zoho.Tasks")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       154416 │
└──────────────┘

In [6]:
duck.sql(
    """
    select t1.Id::varchar, t1.What_Id, t1.Who_id from postgres_db.zoho.Tasks t1
    join (select try_cast(Id as int64) as Id, What_Id, Who_id from read_csv('data/Tasks - Tasks.csv')) t2
    on t1.Id = t2.Id
    offset 100
    """
)

┌────────────────────────┬────────────────────┬────────────────────┐
│ CAST(t1.Id AS VARCHAR) │      What_Id       │       Who_Id       │
│        varchar         │       int64        │       int64        │
├────────────────────────┼────────────────────┼────────────────────┤
│ 386758000027924622     │ 386758000026712960 │               NULL │
│ 386758000027924588     │ 386758000026132416 │               NULL │
│ 386758000027924562     │ 386758000024592512 │               NULL │
│ 386758000027930715     │ 386758000027930560 │ 386758000027930560 │
│ 386758000027924548     │ 386758000024189312 │               NULL │
│ 386758000027924521     │ 386758000023792064 │               NULL │
│ 386758000027920409     │ 386758000022235136 │               NULL │
│ 386758000027924500     │ 386758000023293568 │               NULL │
│ 386758000027924486     │ 386758000023174016 │               NULL │
│ 386758000027932024     │ 386758000017038976 │               NULL │
│         ·              │        

In [9]:
duck.sql("""
select try_cast(Id as int64) as Id, What_Id, Who_id from read_csv('data/Tasks - Tasks.csv')
""")

┌────────────────────┬────────────────────┬────────────────────┐
│         Id         │      What_Id       │       Who_Id       │
│       int64        │      varchar       │      varchar       │
├────────────────────┼────────────────────┼────────────────────┤
│ 386758000061192211 │ 386758000061193001 │ NULL               │
│ 386758000061192185 │ 386758000061192445 │ 386758000061192440 │
│ 386758000061148026 │ 386758000061124693 │ 386758000061124688 │
│ 386758000061151021 │ 386758000060315001 │ NULL               │
│ 386758000061147032 │ 386758000056967806 │ 386758000054274376 │
│ 386758000061181003 │ 386758000054274381 │ 386758000054274376 │
│ 386758000061178003 │ 386758000060590663 │ 386758000060590658 │
│ 386758000061165003 │ 386758000058411726 │ NULL               │
│ 386758000061125003 │ 386758000060763002 │ NULL               │
│ 386758000061146001 │ 386758000060910001 │ NULL               │
│          ·         │         ·          │  ·                 │
│          ·         │   

In [10]:
duck.sql(
    """
    update postgres_db.zoho.Tasks t1
    set What_Id = t2.What_Id,
        Who_id = t2.Who_id
    from (select try_cast(Id as int64) as Id, try_cast(What_Id as int64) as What_Id, try_cast(Who_id as int64) as  Who_id from read_csv('data/Tasks - Tasks.csv')) t2
    where t1.Id = t2.Id
    """
)

In [ ]:
df = duck.sql("select * from read_csv('data/Deals (business opportunities) - Deals.csv')").df()

,Id,Owner,Amount,Deal_Name,Closing_Date,Account_Name,Account Status (Formula Victor),Stage,Type,Probability,...,Kosten_AMED_Brutto_anteilig,Summe_Betreuungskosten_Netto_anteilig,Anfahrtskosten_Info,Details4Lost,Kontaktformular_Standort,Custom_Lead_ID,System_Lead_ID,Record_Creation_Source_ID__s,Connected_To__s.module.api_name,Connected_To__s
0,<NA>,<NA>,NaN,None,NaT,<NA>,None,None,None,<NA>,...,None,NaN,None,None,None,None,<NA>,<NA>,None,None
1,386758000059585264,386758000033113001,51072.0,zentraler Dienst der bayerischen Staatstheater...,2028-08-26,386758000048712593,Active Deal,Angelegt,Vertragsverlängerung,25,...,None,NaN,None,None,München,None,386758000048671001,0,None,None
2,386758000059585243,386758000033113001,51072.0,zentraler Dienst der bayerischen Staatstheater...,2027-08-26,386758000048712593,Active Deal,Angelegt,Vertragsverlängerung,25,...,None,NaN,None,None,München,None,386758000048671001,0,None,None
3,386758000036737386,386758000000341639,2000.0,Erste Abwicklungsanstalt (EAA) - - Große Firma...,2024-01-22,386758000036737301,Active Deal,Verloren,Neues Geschäft,0,...,None,NaN,None,None,Düsseldorf,None,<NA>,0,None,None
4,386758000037615064,386758000000341639,17490.0,Aluminium Oxid Stade GmbH - II - Große Firma A...,2024-02-19,386758000037615056,Active Deal,Verloren,Neues Geschäft,0,...,None,NaN,None,None,Stade-Bützfleth,None,<NA>,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19625,386758000061040048,386758000018517006,31500.0,degewo Aktiengesellschaft_BMP,2025-11-30,386758000010027126,None,Qualifikation,Neues Geschäft,10,...,None,NaN,None,None,None,None,386758000061006955,0,None,None
19626,386758000061111001,386758000017411149,598.0,10_2025_TurbineKreuzberg_ErsthelferkursMentale...,2025-10-06,386758000011736342,None,Gewonnen,Neues Geschäft,100,...,None,NaN,None,None,None,None,<NA>,0,None,None
19627,386758000061115129,386758000018517006,650.0,PISM - - Große Firma ArbMed,2025-11-10,386758000061115121,None,Angebot erstellen,Neues Geschäft,75,...,None,NaN,None,None,Berlin,None,386758000061096002,0,None,None
19628,386758000061115712,386758000018517006,2000.0,deinweiss Zahnärzte - III - Kleine Firma (ArbM...,2025-11-10,386758000061115704,None,Angebot erstellen,Neues Geschäft,75,...,None,NaN,None,None,Gotzkowskystraße 1 10555 Berlin,None,386758000060988002,0,None,None


In [18]:
import pandas as pd

In [24]:
pd.read_csv('data/Deals (business opportunities) - Deals.csv', dtype={'Id': 'Int64', 'Owner': 'Int64', 'Account_Name': 'Int64', 'System_Lead_ID': 'Int64', 'Custom_Lead_ID': 'Int64'})

/var/folders/99/6ltr_n_s7fq3lxwcq9fhbpdw0000gq/T/ipykernel_50968/2024405206.py:1: DtypeWarning: Columns (6,33,53,55,56,61,70,74,75,76,77,112,113,117,119,123,124,126,127,129,130,131,136,139,140,142,143,149,150,151,152,153,154,157,158,159,163,165) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('data/Deals (business opportunities) - Deals.csv', dtype={'Id': 'Int64', 'Owner': 'Int64', 'Account_Name': 'Int64', 'System_Lead_ID': 'Int64', 'Custom_Lead_ID': 'Int64'})


,Id,Owner,Amount,Deal_Name,Closing_Date,Account_Name,Account Status (Formula Victor),Stage,Type,Probability,...,Kosten_AMED_Brutto_anteilig,Summe_Betreuungskosten_Netto_anteilig,Anfahrtskosten_Info,Details4Lost,Kontaktformular_Standort,Custom_Lead_ID,System_Lead_ID,Record_Creation_Source_ID__s,Connected_To__s.module.api_name,Connected_To__s
0,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN
1,386758000059585264,386758000033113001,51072.0,zentraler Dienst der bayerischen Staatstheater...,2028-08-26,386758000048712593,Active Deal,Angelegt,Vertragsverlängerung,25.0,...,NaN,NaN,NaN,NaN,München,<NA>,386758000048671001,0.0,NaN,NaN
2,386758000059585243,386758000033113001,51072.0,zentraler Dienst der bayerischen Staatstheater...,2027-08-26,386758000048712593,Active Deal,Angelegt,Vertragsverlängerung,25.0,...,NaN,NaN,NaN,NaN,München,<NA>,386758000048671001,0.0,NaN,NaN
3,386758000036737386,386758000000341639,2000.0,Erste Abwicklungsanstalt (EAA) - - Große Firma...,2024-01-22,386758000036737301,Active Deal,Verloren,Neues Geschäft,0.0,...,NaN,NaN,NaN,NaN,Düsseldorf,<NA>,<NA>,0.0,NaN,NaN
4,386758000037615064,386758000000341639,17490.0,Aluminium Oxid Stade GmbH - II - Große Firma A...,2024-02-19,386758000037615056,Active Deal,Verloren,Neues Geschäft,0.0,...,NaN,NaN,NaN,NaN,Stade-Bützfleth,<NA>,<NA>,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19625,386758000061040048,386758000018517006,31500.0,degewo Aktiengesellschaft_BMP,2025-11-30,386758000010027126,NaN,Qualifikation,Neues Geschäft,10.0,...,NaN,NaN,NaN,NaN,NaN,<NA>,386758000061006955,0.0,NaN,NaN
19626,386758000061111001,386758000017411149,598.0,10_2025_TurbineKreuzberg_ErsthelferkursMentale...,2025-10-06,386758000011736342,NaN,Gewonnen,Neues Geschäft,100.0,...,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,0.0,NaN,NaN
19627,386758000061115129,386758000018517006,650.0,PISM - - Große Firma ArbMed,2025-11-10,386758000061115121,NaN,Angebot erstellen,Neues Geschäft,75.0,...,NaN,NaN,NaN,NaN,Berlin,<NA>,386758000061096002,0.0,NaN,NaN
19628,386758000061115712,386758000018517006,2000.0,deinweiss Zahnärzte - III - Kleine Firma (ArbM...,2025-11-10,386758000061115704,NaN,Angebot erstellen,Neues Geschäft,75.0,...,NaN,NaN,NaN,NaN,Gotzkowskystraße 1 10555 Berlin,<NA>,386758000060988002,0.0,NaN,NaN
